# SimHash
To derive the probability that a SimHash value collides for two different vectors despite them having different underlying data, we consider the geometry involved in the hashing process. The critical factor is the angle $\theta$ between the two vectors in question.

### Background

SimHash uses random hyperplanes (defined by random vectors) to divide the space. The side of the hyperplane on which a vector falls determines each bit of its hash. The angle $\theta$ between two vectors affects the probability that they will be hashed to the same side of a randomly chosen hyperplane.

### Collision Probability Derivation

1. **Hyperplane Projection**: Consider a hyperplane defined by a normal vector $Z$. When we project two vectors $X$ and $Y$ onto this hyperplane, the projection's sign determines the hash bit. The hash bit is $1$ if the dot product $X \cdot Z > 0$ and $-1$ otherwise. This hyperplane separates the plane into two areas.

2. **Angle Between Vectors**: The angle $\theta$ between $X$ and $Y$ can be related to their dot product through the cosine similarity formula:

   $$
   \cos(\theta) = \frac{X \cdot Y}{\|X\|\|Y\|}
   $$

3. **Probability of Different Hash Bits**: The probability that $X$ and $Y$ have different hash bits for a single projection is determined by the angle $\theta$ they form, because it affects whether they fall on the same or opposite sides of the hyperplane. This is given by the fraction of the circle's circumference that corresponds to angles leading to different signs in the projection:

   $$
   P(\text{different hash bits}) = \frac{\theta}{\pi}
   $$

   since the hyperplane can be oriented in any direction, and $\theta$ effectively measures the "arc" over the unit circle where the projections of $X$ and $Y$ would have different signs.

4. **Probability of Collision**: The probability that $X$ and $Y$ have the same hash bit for a single projection (a collision) is the complement of them having different hash bits:

   $$
   P(\text{collision}) = 1 - P(\text{different hash bits}) = 1 - \frac{\theta}{\pi} \approx \frac{\cos(\theta) + 1}{2}
   $$

From this, we can say that Simhash approximates cosine similarity.

### Conclusion

The derived equation $P(\text{collision}) = 1 - \frac{\theta}{\pi}$ quantifies the probability that two vectors will be hashed to the same value for a single bit in their SimHash, based solely on the angle $\theta$ between them. This highlights the geometric basis of SimHash: vectors that are closer together (smaller $\theta$) are more likely to collide, which is useful for identifying similar items in applications like duplicate detection or near-duplicate document retrieval.


<img src="simhash.png" alt="simhash">

source: https://people.cs.umass.edu/~cmusco/CS514S20/slides/lecture7/lecture7Compressed.pdf

<img src="simhash_approx.png" alt="simhash_approx">

In [59]:
import numpy as np

def simhash_single(Z, X):
    """
    Generate a SimHash for input vector X using a given random vector Z.
    
    Parameters:
    - Z: numpy array, random vector used for hashing.
    - X: numpy array, document vector.
    
    Returns:
    - int, SimHash value as an integer.
    """
    # Calculate the hash: h(X) = sign(X^T Z)
    assert Z.shape[0] == 1, "Z must be a row vector"
    assert X.shape[0] == 1, "X must be a row vector"
    assert Z.shape[1] == X.shape[1], "Z and X must have the same number of columns"

    hash_bit = np.sign(np.dot(Z, X.T))
    hash_bit = hash_bit[0, 0]
    return hash_bit

# Example usage
p = 1000
X1 = np.random.rand(1, p)  # Simulate a document vector
X2 = X1 + np.random.normal(0, 0.1, (1, p))  # Simulate a similar document vector
X3 = -X1  # Simulate a dissimilar document vector

# Generate a single random vector Z to be used for all documents
d = 32
Z = np.random.normal(0, 1, (1, p))

hash1 = simhash_single(Z, X1)
hash2 = simhash_single(Z, X2)
hash3 = simhash_single(Z, X3)

print("Original document hash:", hash1)
print("Similar document hash:", hash2)
print("Dissimilar document hash:", hash3)

Original document hash: -1.0
Similar document hash: -1.0
Dissimilar document hash: 1.0



### Algorithm Overview

The goal is to create a SimHash function that minimizes collisions between dissimilar keys while allowing similar keys to hash to the same or similar hash values. Using multiple hash functions (via a random matrix $Z$) increases the reliability of this process. The steps to compute SimHash for a dataset $X$ are as follows:

1. **Dataset $X$**: Consider your dataset $X$ where $X \in \mathbb{R}^{n \times p}$. Here, $n$ is the number of data points (or vectors), and $p$ is the dimensionality of each vector.

2. **Random Matrix $Z$**: Generate a random matrix $Z \in \mathbb{R}^{k \times p}$ where each element $Z_{i, j}$ is drawn from a normal distribution $\mathcal{N}(0, 1)$. 

3. **Hash Computation $H$**: Compute the hash matrix $H$ as $H = \text{sign}(XZ^T)$. The sign function is applied element-wise, producing a matrix $H \in \{ -1, 1 \}^{n \times k}$, where each row corresponds to the hash value of a data point in $X$.

4. **Hash Value Interpretation**: Convert each row $H[i]$ of $H$ into an integer hash value. This can be done by interpreting each row as a binary number, where $-1$ maps to $0$ and $1$ maps to $1$, and then converting this binary representation to an integer. For example, the binary representation $[1, -1, 1, -1]$ (or equivalently $[1, 0, 1, 0]$ after mapping) corresponds to the integer $10$ in binary, or $2$ in decimal.


In [61]:
import numpy as np

def generate_random_hyperplanes(num_bits, dim):
    """
    Generate random hyperplanes.
    
    Parameters:
    - num_bits: The number of bits (or features) in the SimHash.
    - dim: The dimensionality of the input vectors.
    
    Returns:
    - A numpy array of shape (num_bits, dim) representing random hyperplanes.
    """
    return np.random.randn(num_bits, dim)

def simhash_for_dataset(dataset, num_bits=64):
    """
    Compute the SimHash for each vector in the dataset.
    
    Parameters:
    - dataset: A numpy array of shape (num_vectors, dim) representing the input dataset.
    - num_bits: The number of bits in the SimHash.
    
    Returns:
    - A list of integers representing the SimHash values for the dataset.
    """
    num_vectors, dim = dataset.shape
    hyperplanes = generate_random_hyperplanes(num_bits, dim)
    simhash_values = []
    
    for vector in dataset:
        hash_bits = np.sign(np.dot(hyperplanes, vector))  # Project vector onto each hyperplane
        hash_bits = (hash_bits > 0).astype(int)  # Convert to 0s and 1s
        # Convert binary array to a single integer
        hash_value = sum(1 << i for i, bit in enumerate(hash_bits[::-1]) if bit)
        simhash_values.append(hash_value)
    
    return simhash_values

# Example usage
p = 100  # Dimensionality of the input vectors
n = 10   # Number of vectors in the dataset
dataset = np.random.rand(n, p)  # Simulate a random dataset

# Compute SimHash values
simhash_values = simhash_for_dataset(dataset, num_bits=64)

for i, hash_value in enumerate(simhash_values):
    print(f"Vector {i}: SimHash = {hash_value}")


Vector 0: SimHash = 6765109592584607344
Vector 1: SimHash = 2083546315970037409
Vector 2: SimHash = 5540586791801837297
Vector 3: SimHash = 2082473222285252465
Vector 4: SimHash = 6838187683736934001
Vector 5: SimHash = 6767307549748680545
Vector 6: SimHash = 2082667844299118113
Vector 7: SimHash = 423372856053028385
Vector 8: SimHash = 6730277064216639073
Vector 9: SimHash = 2063316430724748841


Choosing the number of output bits $k$ in SimHash (i.e., the length of the SimHash value) and specifying a similarity threshold for detecting near-duplicates involves a balance between the precision and recall of the duplicate detection process, as well as computational considerations. Here's how to approach these decisions:

### Deciding on $k$ (The Number of Output Bits)

1. **Trade-off Consideration**: A larger $k$ (more bits in the SimHash) allows for finer granularity in distinguishing between items, potentially reducing false positives (dissimilar items being considered similar). However, it also increases the chance of false negatives (failing to identify similar items as such), as even small differences will result in different hashes.

2. **Storage and Computation**: More bits mean more storage and potentially more computational overhead, especially when comparing hashes or searching for near-duplicates within large datasets.

3. **Empirical Testing**: Often, the choice of $k$ is empirically determined based on the specific requirements of the application, including acceptable levels of false positives/negatives and computational resources. Common values for $k$ are 64, 128, or 256 bits, providing a balance between specificity and computational efficiency.

### Specifying Similarity Thresholds and Search Ranges

1. **Hamming Distance for Similarity**: The similarity between two SimHash values is often measured using the Hamming distance, which counts the number of bits that differ between them. A lower Hamming distance indicates higher similarity.

2. **Threshold Setting**: You can set a threshold on the Hamming distance to decide whether two items are considered "similar." For example, with $k = 64$, you might decide that items with a Hamming distance of 3 or less are similar. This threshold depends on your specific needs for precision (how many of the retrieved items are relevant) and recall (how many of the relevant items are retrieved).

3. **Adjusting for Specific Similarity Values**: If you have a specific similarity value in mind (e.g., wanting to detect items that are at least 90% similar), you can relate this to the Hamming distance by considering the percentage of bits that are the same. For $k = 64$ bits, at least 90% similarity means at most 6 bits differ (since $64 \times 0.1 = 6.4$). Thus, your threshold for the Hamming distance would be 6 to achieve at least 90% similarity.

4. **Search Range and Neighboring Hash Cells**: When searching for near-duplicates in a dataset, you can use the similarity threshold to limit your search to items within a certain Hamming distance. Efficient searching can be done using data structures optimized for Hamming space, such as BK-trees, or by leveraging locality-sensitive hashing techniques to group items likely to be similar.

### Implementation Considerations

When implementing SimHash and deciding on $k$ and similarity thresholds, consider the following:

- **Pilot Testing**: Run pilot tests on subsets of your data with different values of $k$ and similarity thresholds to assess the impact on precision, recall, and computational resources.
- **Dynamic Adjustments**: In some applications, it might be beneficial to adjust $k$ and the similarity threshold dynamically based on ongoing performance evaluation and the evolving needs of the application.

Choosing $k$ and specifying similarity thresholds are critical steps that should be tailored to the specific requirements and constraints of your application, balancing accuracy and computational efficiency.

# Connection With Hoeffding's Inequality

Hoeffding's Inequality is a statistical tool that can provide bounds on the probability that the sum of random variables deviates from its expected value. It's often used in probability theory and statistics to make inferences about the sum of bounded independent random variables. While Hoeffding's Inequality is not directly used to decide the length $k$ of a SimHash output, the principles underlying it can guide the selection of $k$ by considering the reliability and variance of the hash values as a form of statistical estimation.

### Understanding Hoeffding's Inequality

Hoeffding's Inequality gives an upper bound on the probability that the sum of independent random variables deviates from its expected value by more than a certain amount. Specifically, for independent bounded random variables, it helps quantify the confidence in their aggregate behavior.

#### Hoeffding's Inequality

Hoeffding's Inequality gives us a powerful tool to estimate the probability of large deviations for the sum of independent bounded random variables. Specifically, for $k$ independent random variables $X_1, X_2, ..., X_k$ with bounds $a_i \leq X_i \leq b_i$, the sum $S_k = \sum_{i=1}^{k} X_i$ satisfies:

$$
\Pr\left(|S_k - \mathbb{E}[S_k]| \geq t\right) \leq 2\exp\left(-\frac{2t^2}{\sum_{i=1}^{k}(b_i - a_i)^2}\right)
$$

for any $t > 0$.

If $X_i \in [0, 1/k]$, then the inequality becomes

$$
\Pr\left(|S_k - \mathbb{E}[S_k]| \geq t\right) \leq 2\exp\left(-\frac{2t^2}{\sum_{i=1}^{k}k^2}\right) = 2\exp\left(-2t^2k\right) 
$$

### Applying the Concept to SimHash

When applying concepts similar to those in Hoeffding's Inequality to decide $k$ for SimHash, we are essentially concerned with two aspects:

1. **Reliability of Hash Values**: Increasing $k$ (the number of bits in the SimHash) increases the reliability of the hash to represent the document because more bits mean more random projections, leading to a finer granularity in capturing the document's features.

2. **Variance and Confidence**: With more bits, the variance in how well the SimHash represents any given document decreases, and our confidence in the hash value as a stable identifier of the document's content increases. This concept parallels the idea of reducing the bound on deviation (as per Hoeffding's Inequality), thus increasing our confidence in the statistical estimations made from the hash values.

### Practical Considerations

1. **Choosing $k$ for Desired Confidence**: In practice, while Hoeffding's Inequality itself may not prescribe a specific value of $k$, the underlying idea of balancing the expected accuracy (confidence) against the probability of error can inform the choice. For instance, a larger $k$ would decrease the likelihood that two non-similar documents produce a similar hash (reducing false positives) but increases computational and storage requirements.

2. **Error Bounds and Similarity Thresholds**: Similarly, when determining similarity thresholds or deciding how closely two SimHash values must match to be considered duplicates, considerations similar to those in applying Hoeffding's Inequality can help. Specifically, understanding the expected variance in hash values due to the randomness of the projections and choosing thresholds that balance the risk of false positives/negatives with the need for computational efficiency.

### Conclusion

While Hoeffding's Inequality itself isn't directly used to decide $k$ in SimHash, the principles of statistical confidence and bounding errors it embodies can guide the choice of $k$. By considering the trade-offs between the reliability of the SimHash values, the computational resources available, and the desired level of confidence in distinguishing documents, one can make an informed decision on the appropriate length of the SimHash output.

# Limitation of Simhash

* Simhash uses approximation of approximation: $P(h(x) = h(y)) \approx (cos(\theta) + 1) / 2 \approx distance
* Numerical computing was difficult in 1990s.